In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import pickle

In [4]:
X_train = pickle.load(open('../Data/X_train.pkl', 'rb'))
y_train = pickle.load(open('../Data/y_train.pkl', 'rb'))
X_test = pickle.load(open('../Data/X_test.pkl', 'rb'))
y_test = pickle.load(open('../Data/y_test.pkl', 'rb'))

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(329, 20530)
(329,)
(83, 20530)
(83,)


In [5]:
# standardize the data
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

print(X_train)

[[-0.48855657  0.12112708 -0.33372349 ...  0.04735987  0.50779996
  -0.02217494]
 [ 0.83092164  1.17948768 -0.33372349 ...  0.18754437  0.77965228
  -0.17205193]
 [-1.861054   -0.69352094 -0.33372349 ... -0.35346398 -2.00005787
   3.63814136]
 ...
 [ 1.50287947  0.88173571  0.1048791  ... -0.33180584  0.91213697
  -0.14253948]
 [-0.06583194  0.60359346 -0.33372349 ...  0.20030191  0.44110666
  -1.31512921]
 [-0.87784052 -0.01478846 -0.33372349 ...  0.12805867  1.46177518
  -0.6430939 ]]


In [6]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device

device(type='cpu')

In [7]:
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

(torch.Size([329, 20530]),
 torch.Size([83, 20530]),
 torch.Size([329]),
 torch.Size([83]))

In [10]:
from sklearn.model_selection import KFold

kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# kfold.split() returns indices to split data into training and test set
for train_idx, test_idx in kfold.split(X_train, y_train):
    print(train_idx, test_idx)
    print(len(train_idx), len(test_idx))

[  0   1   2   4   6   8  10  11  12  13  14  15  17  18  19  20  21  22
  23  24  26  27  28  29  30  31  32  34  35  36  37  38  39  40  41  43
  44  47  48  49  50  51  52  53  54  56  58  59  61  62  64  65  66  67
  68  69  70  71  72  74  75  76  79  80  81  83  85  86  87  88  89  91
  92  93  95  96  97  98  99 100 102 103 104 105 106 107 108 109 112 113
 114 115 116 117 118 120 121 122 123 125 126 127 128 129 130 131 132 133
 134 135 136 138 141 142 143 146 148 149 150 151 152 153 154 155 156 157
 158 159 160 161 162 165 166 167 169 170 171 174 178 179 180 181 183 184
 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202
 203 204 205 206 207 208 209 211 212 213 214 215 216 217 218 220 221 223
 224 225 226 227 228 229 230 231 233 234 235 236 238 239 240 241 242 243
 244 245 246 247 248 249 251 252 253 254 255 257 258 259 261 263 264 265
 266 267 268 269 270 272 273 276 278 279 280 281 282 283 284 285 286 287
 290 291 292 293 294 295 298 299 300 302 303 304 30

## Define the model

In [11]:
class DLModel(nn.Module):
    def __init__(self, input_size):
        super(DLModel, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 1024),
            nn.ReLU(),
            nn.Linear(1024, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.layers(x)

In [16]:
# model parameters
input_size = 20530
learning_rate = 0.01
epochs = 100
batch_size = 32
n_splits = 5

In [19]:
from tqdm import tqdm

# Instantiate the model
model = DLModel(input_size).to(device)

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
for epoch in tqdm(range(epochs)):
    for i in range(0, len(X_train), batch_size):
        X_batch = X_train[i:i+batch_size]
        y_batch = y_train[i:i+batch_size]
        
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch.view(-1, 1))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# test accuracy
with torch.no_grad():
    y_pred = model(X_test)
    y_pred = y_pred.round()
    acc = y_pred.eq(y_test.view_as(y_pred)).sum() / float(len(y_test))
    print(f'Test accuracy: {acc:.4f}')

# test auc
with torch.no_grad():
    y_pred = model(X_test)
    y_pred = y_pred.round()
    auc = roc_auc_score(y_test, y_pred)
    print(f'Test AUC: {auc:.4f}')

100%|██████████| 100/100 [05:03<00:00,  3.03s/it]

Test accuracy: 0.9157
Test AUC: 0.8764


## Define a little deeper model

In [34]:
class DLModel(nn.Module):
    def __init__(self, input_size):
        super(DLModel, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 1024),
            nn.ReLU(),
            nn.Linear(1024, 256),
            nn.ReLU(),
            nn.Linear(256, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.layers(x)

In [35]:
from tqdm import tqdm

# Instantiate the model
model = DLModel(input_size).to(device)

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
for epoch in tqdm(range(epochs)):
    for i in range(0, len(X_train), batch_size):
        X_batch = X_train[i:i+batch_size]
        y_batch = y_train[i:i+batch_size]
        
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch.view(-1, 1))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# test accuracy
with torch.no_grad():
    y_pred = model(X_test)
    y_pred = y_pred.round()
    acc = y_pred.eq(y_test.view_as(y_pred)).sum() / float(len(y_test))
    print(f'Test accuracy: {acc:.4f}')

# test auc
with torch.no_grad():
    y_pred = model(X_test)
    y_pred = y_pred.round()
    auc = roc_auc_score(y_test, y_pred)
    print(f'Test AUC: {auc:.4f}')

100%|██████████| 100/100 [05:42<00:00,  3.43s/it]

Test accuracy: 0.8916
Test AUC: 0.8377


## Define a deeeep model!

In [30]:
class DLModel(nn.Module):
    def __init__(self, input_size):
        super(DLModel, self).__init__()
        self.layers = nn.Sequential(
            nn.Linear(input_size, 1024),
            nn.ReLU(),
            nn.Linear(1024, 1024),
            nn.ReLU(),
            nn.Linear(1024, 256),
            nn.ReLU(),
            nn.Linear(256, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.layers(x)

In [31]:
from tqdm import tqdm

# Instantiate the model
model = DLModel(input_size).to(device)

# Define loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
for epoch in tqdm(range(epochs)):
    for i in range(0, len(X_train), batch_size):
        X_batch = X_train[i:i+batch_size]
        y_batch = y_train[i:i+batch_size]
        
        y_pred = model(X_batch)
        loss = criterion(y_pred, y_batch.view(-1, 1))
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# test accuracy
with torch.no_grad():
    y_pred = model(X_test)
    y_pred = y_pred.round()
    acc = y_pred.eq(y_test.view_as(y_pred)).sum() / float(len(y_test))
    print(f'Test accuracy: {acc:.4f}')

# test auc
with torch.no_grad():
    y_pred = model(X_test)
    y_pred = y_pred.round()
    auc = roc_auc_score(y_test, y_pred)
    print(f'Test AUC: {auc:.4f}')

100%|██████████| 100/100 [06:32<00:00,  3.92s/it]


Test accuracy: 0.9036
Test AUC: 0.8927
